In [123]:
import numpy as np
from numpy import mean
from numpy import std
import pandas as pd
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import xgboost as xgb

## Final pre-processing of data

In [124]:
allScens = pd.read_csv('../Output/allScens_wRandomness.csv', usecols=lambda x: x not in ['Unnamed: 0'])

In [125]:
allScens.head()

,City,Exist_Fuel_Type,Scenario,Census_Area,ANCSA_Region,Util_Name,PCE,Sq_Ft,Capital_Cost,Elec_Use_Jan,...,Avg_Temp_3,Avg_Temp_4,Avg_Temp_5,Avg_Temp_6,Avg_Temp_7,Avg_Temp_8,Avg_Temp_9,Avg_Temp_10,Avg_Temp_11,Avg_Temp_12
0,Adak,1,Base,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1289.367468,5596.353187,430.689472,...,29.256,29.997,31.818,35.787,39.544,40.967,40.145,35.104,30.510,27.509
1,Adak,1,Small Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,773.620481,5596.353187,344.551577,...,29.211,29.950,31.768,35.731,39.483,40.903,40.082,35.049,30.463,27.466
2,Adak,1,Large Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1934.051202,5596.353187,430.689472,...,37.992,38.954,41.319,46.473,51.352,53.200,52.132,45.586,39.621,35.723
3,Adak,1,Low Elec,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1289.367468,5596.353187,301.482630,...,43.818,44.927,47.655,53.599,59.227,61.358,60.127,52.577,45.696,41.201
4,Adak,1,No PCE,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1289.367468,5596.353187,430.689472,...,35.293,36.186,38.383,43.171,47.703,49.420,48.428,42.347,36.805,33.185


In [188]:
allScens = pd.get_dummies(allScens, columns=['Exist_Fuel', 'Census_Area', 'Scenario'])

KeyError: "['Exist_Fuel', 'Census_Area', 'Scenario'] not in index"

In [127]:
allScens.columns

Index(['City', 'Exist_Fuel_Type', 'ANCSA_Region', 'Util_Name', 'PCE', 'Sq_Ft',
       'Capital_Cost', 'Elec_Use_Jan', 'Elec_Use_May', 'Rebate_dol',
       'Fuel_Esc_Rate', 'Exist_Unit_Fuel_Cost', 'Design_Heat_Load',
       'Design_Heat_Temp', 'COP', 'Max_HP_Cap_Reached', 'HP_Load_Frac',
       'Avg_Temp', 'Freezing_days', 'IRR', 'NPV', 'CO2_lbs_saved',
       'CO2_driving_miles_saved', 'Fuel_Use_Chg', 'Fuel_Price_Incremental',
       'Elec_Use_Chg', 'Elec_Rate_Incremental', 'Elec_Rate_Avg_Base', 'Econ',
       'Avg_Temp_1', 'Avg_Temp_2', 'Avg_Temp_3', 'Avg_Temp_4', 'Avg_Temp_5',
       'Avg_Temp_6', 'Avg_Temp_7', 'Avg_Temp_8', 'Avg_Temp_9', 'Avg_Temp_10',
       'Avg_Temp_11', 'Avg_Temp_12', 'Exist_Fuel_#1 Oil',
       'Exist_Fuel_Electricity', 'Exist_Fuel_Natural Gas',
       'Exist_Fuel_Propane', 'Census_Area_Aleutians East Borough',
       'Census_Area_Aleutians West Census Area',
       'Census_Area_Anchorage municipality', 'Census_Area_Bethel Census Area',
       'Census_Area_Bris

## Building regression models

In [181]:
# Separating the df into input and output components
allScens_numerics1 = allScens.filter(regex = 'Exist_Fuel_(?!Type)|Avg_Temp_[1,2,3,10,11,12]|Elec_Use_')
allScens_numerics2 = allScens[['Freezing_days', 'Exist_Unit_Fuel_Cost', 'Elec_Rate_Avg_Base', 'PCE', 'Sq_Ft', 'Capital_Cost', 'Design_Heat_Load', 'Design_Heat_Temp', 'Rebate_dol', 'Fuel_Esc_Rate']]
allScens_numerics3 = pd.concat([allScens['Fuel_Esc_Rate']**2, allScens['Freezing_days']**2, allScens['Rebate_dol']**2], axis=1)
allScens_numerics3.rename(columns={'Fuel_Esc_Rate':'Fuel_Esc_Rate2', 'Freezing_days':'Freezing_days2', 'Rebate_dol':'Rebate_dol2'}, inplace=True)

X = pd.concat([allScens_numerics1, allScens_numerics2, allScens_numerics3], axis=1)
Y = allScens['NPV']
cities = allScens['City']

In [182]:
X.head()

,Elec_Use_Jan,Elec_Use_May,Elec_Use_Chg,Avg_Temp_1,Avg_Temp_2,Avg_Temp_3,Avg_Temp_10,Avg_Temp_11,Avg_Temp_12,Exist_Fuel_#1 Oil,...,PCE,Sq_Ft,Capital_Cost,Design_Heat_Load,Design_Heat_Temp,Rebate_dol,Fuel_Esc_Rate,Fuel_Esc_Rate2,Freezing_days2,Rebate_dol2
0,430.689472,388.445152,-8906.176992,26.706,27.833,29.256,35.104,30.510,27.509,0,...,0.7597,1289.367468,5596.353187,13625.603079,22.6,5668.906232,0.064799,0.004199,0.209764,3.213650e+07
1,344.551577,310.756122,-1348.134275,26.664,27.790,29.211,35.049,30.463,27.466,0,...,0.7597,773.620481,5596.353187,9005.232119,22.6,642.754715,0.086262,0.007441,0.217156,4.131336e+05
2,430.689472,388.445152,-10452.336399,34.680,36.144,37.992,45.586,39.621,35.723,0,...,0.7597,1934.051202,5596.353187,19401.066778,22.6,4121.830264,0.071862,0.005164,0.008100,1.698948e+07
3,301.482630,271.911606,-14294.891308,39.999,41.687,43.818,52.577,45.696,41.201,0,...,0.7597,1289.367468,5596.353187,13625.603079,22.6,1272.883949,0.106926,0.011433,0.000484,1.620234e+06
4,430.689472,388.445152,-15339.844898,32.216,33.576,35.293,42.347,36.805,33.185,0,...,0.7597,1289.367468,5596.353187,13625.603079,22.6,2660.864153,0.115070,0.013241,0.017424,7.080198e+06


In [175]:
# Compare to the mean
np.mean(allScens['NPV'])

33683.30427644264

## Group-based split

In [183]:
#from sklearn.model_selection import GroupShuffleSplit 
# splitter = GroupShuffleSplit(test_size=.33, n_splits=2, random_state = 7)
# split = splitter.split(allScens, groups=allScens['City'])
# train_inds, test_inds = next(split)

from sklearn.model_selection import GroupKFold 
split = GroupKFold(n_splits=5).split(allScens, groups=allScens['City'])
train_inds, test_inds = next(split)

In [184]:
len(train_inds)

15800

In [185]:
len(test_inds)

3945

In [186]:
X_train = X.iloc[train_inds]
Y_train = Y.iloc[train_inds]
X_test = X.iloc[test_inds]
Y_test = Y.iloc[test_inds]

## Extreme Gradient boosting model (XGB)

In [187]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

# Naive model
xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, Y_train)

# Making predictions
Y_pred = xgb_reg.predict(X_test)

# Assess performance
rmse = mean_squared_error(Y_test, Y_pred, squared=False)
mae = mean_absolute_error(Y_test, Y_pred)
print("The RMSE of the model is", rmse)
print("The MAE of the model is", mae)

The RMSE of the model is 17396.084852462114
The MAE of the model is 10605.087074018535


### Random Search CV

In [73]:
params = {
 'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
 'n_estimators': [50, 100, 200, 500],
 'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
 'subsample' : [0.2, 0.4, 0.8, 1.0],
 'min_child_weight' : [ 1, 3, 5, 7 ],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [74]:
xgb_reg = xgb.XGBRegressor()

In [75]:
cv = GroupKFold(n_splits=5)
rs_model=RandomizedSearchCV(xgb_reg,
                            param_distributions=params,
                            n_iter=50,
                            cv=cv,
                            scoring='neg_mean_absolute_error',
                            n_jobs=-1,
                            verbose=2)

In [76]:
rs_model.fit(X_train, Y_train, groups = cities.iloc[train_inds])

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[CV] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=7, n_estimators=500, subsample=0.2; total time=   7.0s
[CV] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=7, n_estimators=500, subsample=0.2; total time=   6.9s
[CV] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=7, n_estimators=500, subsample=0.2; total time=   7.0s
[CV] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=7, n_estimators=500, subsample=0.2; total time=   7.1s
[CV] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.3, max_depth=5, min_child_weight=7, n_estimators=500, subsample=0.2; total time=   7.0s
[CV] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.25, max_depth=15, min_child_weight=3, n_estimators=50, subsample=0.8; total time=   3.5s
[CV] END colsample_bytree=0.4, gamma=0.1, learning_rate=0.25, max_depth=15, min_child_weight=3, n_estimators=50, subsample=

/Users/brianleung/miniconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END colsample_bytree=0.7, gamma=0.4, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=50, subsample=1.0; total time=   1.8s
[CV] END colsample_bytree=0.4, gamma=0.2, learning_rate=0.05, max_depth=15, min_child_weight=5, n_estimators=500, subsample=1.0; total time=  29.9s
[CV] END colsample_bytree=0.7, gamma=0.4, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=50, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.7, gamma=0.4, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=50, subsample=1.0; total time=   1.8s
[CV] END colsample_bytree=0.7, gamma=0.4, learning_rate=0.15, max_depth=6, min_child_weight=3, n_estimators=50, subsample=1.0; total time=   1.9s
[CV] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7, n_estimators=100, subsample=1.0; total time=   1.3s
[CV] END colsample_bytree=0.3, gamma=0.2, learning_rate=0.15, max_depth=5, min_child_weight=7, n_estimators=100, subsampl

RandomizedSearchCV(cv=GroupKFold(n_splits=5),
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraint...
                                          predictor=None, random_state=None, ...),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5,
                                                             0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2,
                                                          0.25, 0.3],
                                        'max_depth': [3, 4, 5, 6, 8, 10, 12,
                                                      15],
                                        'min_child_weight': [1, 3, 5, 7],
                                        'n_estimators': [50, 100, 200, 500],
                                        'subsample': [0.2, 0.4, 0.8, 1.0]},
                   scoring='neg_mean_absolute_error', verbose=2)

In [77]:
rs_model.best_params_

{'subsample': 1.0,
 'n_estimators': 500,
 'min_child_weight': 7,
 'max_depth': 8,
 'learning_rate': 0.15,
 'gamma': 0.0,
 'colsample_bytree': 0.7}

In [78]:
best_model = rs_model.best_estimator_

In [79]:
Y_pred = best_model.predict(X_test)
mean_absolute_error(Y_test, Y_pred)

9540.84480758613

In [80]:
cv = GroupKFold(n_splits=5)
score=cross_val_score(best_model,X,Y,cv=cv,groups=cities,scoring='neg_mean_absolute_error')

In [81]:
score.mean()

-10477.075601656821

## Random Forest

In [118]:
# Fitting a random forest 
model = RandomForestRegressor()
model.fit(X_train, Y_train)

# Making predictions
Y_pred = model.predict(X_test)

# Assess performance
rmse = mean_squared_error(Y_test, Y_pred, squared=False)
mae = mean_absolute_error(Y_test, Y_pred)
print("The RMSE of the model is", rmse)
print("The MAE of the model is", mae)

The RMSE of the model is 16983.690714548284
The MAE of the model is 10116.871163068477


### Random Forest -- Random Search CV

In [ ]:
n_estimators = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
min_samples_leaf =  [1, 2, 4]
min_samples_split = [2, 5, 10]
max_features = ['sqrt', 'log2', None]

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'max_features': max_features}


In [ ]:
hyperparameter_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'max_features': ['sqrt', 'log2']}

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [ ]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=hyperparameter_grid, n_iter=50, cv=5, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, Y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1400; total time=  11.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1400; total time=  12.3s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1400; total time=  12.6s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=400; total time=   4.0s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=  16.6s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=  16.8s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=1800; total time=  17.1s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_est

/Users/brianleung/miniconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=70, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=1400; total time=  22.0s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=  22.9s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=  23.1s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=  23.0s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=  22.2s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=  24.5s
[CV] END max_depth=80, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=2000; total time=  24.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=1600; total time=  22.1s
[CV] END max_depth=20, max_

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [ ]:
rf_random.best_params_

{'n_estimators': 1800,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 40}

In [ ]:
best_rf = RandomForestRegressor(**rf_random.best_params_)

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=5)
scores = cross_val_score(best_rf, X, Y, cv = cv, scoring='neg_mean_absolute_error')

In [ ]:
scores.mean()

-3985.506493899842

## Gradient boosting model 

In [120]:
# Fitting a gradient boosting model 
model = GradientBoostingRegressor(**rs_model.best_params_)

TypeError: GradientBoostingRegressor.__init__() got an unexpected keyword argument 'min_child_weight'

In [112]:
# K-fold cross validation 
cv = RepeatedKFold(n_splits=3, n_repeats=3)
n_scores = cross_val_score(model, X, Y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: -2551.385 (69.356)


In [64]:
n_estimators = [100, 500, 1000, 1500]
max_depth = [3, 5, 10, 15]
min_samples_leaf = [2, 4, 6, 10] 
min_samples_split = [2, 4, 6, 10]
max_features = ['auto', 'sqrt']

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'max_features': max_features}

In [65]:
random_cv = RandomizedSearchCV(estimator=model,
                               param_distributions=hyperparameter_grid,
                               cv=4, n_iter=10,
                               scoring = 'neg_mean_absolute_error',n_jobs = -1,
                               verbose = 2, 
                               random_state=42)

In [66]:
random_cv.fit(X, Y)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] END max_depth=10, max_features=auto, min_samples_leaf=10, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=10, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=10, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=10, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_leaf=10, min_samples_split=4, n_estimators=1500; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_leaf=10, min_samples_split=4, n_estimators=1500; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_leaf=10, min_samples_split=4, n_estimators=1500; total time=   0.0s
[CV] END max_depth=15, max_features=auto, min_samples_leaf=10, min_samples_split=4, n_est

/Users/brianleung/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
16 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/brianleung/miniconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/brianleung/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/brianleung/miniconda3/lib/python3.11/site-packages/sklearn/base.py", line 637, in _validate_params
    validate_parameter_const

RandomizedSearchCV(cv=4,
                   estimator=GradientBoostingRegressor(max_depth=5,
                                                       max_leaf_nodes=3,
                                                       n_estimators=1000),
                   n_jobs=-1,
                   param_distributions={'max_depth': [3, 5, 10, 15],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [2, 4, 6, 10],
                                        'min_samples_split': [2, 4, 6, 10],
                                        'n_estimators': [100, 500, 1000, 1500]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [68]:
random_cv.best_params_

{'n_estimators': 1500,
 'min_samples_split': 4,
 'min_samples_leaf': 10,
 'max_features': 'sqrt',
 'max_depth': 5}

In [71]:
best_model = GradientBoostingRegressor(n_estimators= 1500,
                                       min_samples_split= 4,
                                       min_samples_leaf= 10,
                                       max_features='sqrt',
                                       max_depth=5)

score=cross_val_score(best_model,X,Y,cv=3, scoring='neg_mean_absolute_error')

In [72]:
score

array([-8792.47329762, -8229.639407  , -9513.83353661])

## Grid search for hyper-parameters

In [509]:
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot

model = GradientBoostingRegressor()
# define the grid of values to search
grid = dict()
grid['n_estimators'] = [100, 250, 500, 1000]
grid['learning_rate'] = [0.01, 0.1, 0.5, 1]
grid['subsample'] = [0.5, 0.7, 1.0]
grid['max_depth'] = [3, 7, 9]

# define the evaluation procedure
cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='neg_mean_absolute_error')
# execute the grid search
grid_result = grid_search.fit(X, Y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

This version of python seems to be incorrectly compiled
(internal generated filenames are not absolute).
This may make the debugger miss breakpoints.
Related bug: http://bugs.python.org/issue1666807
This version of python seems to be incorrectly compiled
(internal generated filenames are not absolute).
This may make the debugger miss breakpoints.
Related bug: http://bugs.python.org/issue1666807
This version of python seems to be incorrectly compiled
(internal generated filenames are not absolute).
This may make the debugger miss breakpoints.
Related bug: http://bugs.python.org/issue1666807
This version of python seems to be incorrectly compiled
(internal generated filenames are not absolute).
This may make the debugger miss breakpoints.
Related bug: http://bugs.python.org/issue1666807
This version of python seems to be incorrectly compiled
(internal generated filenames are not absolute).
This may make the debugger miss breakpoints.
Related bug: http://bugs.python.org/issue1666807
This 